In [1]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


import networkx as nx
import matplotlib.pyplot as plt

# Parámetros para las redes
num_nodos = [1000, 3000]  # Número de nodos
grados_medios = [2, 4, 8]  # Grado medio de cada nodo

# Almacenar las redes generadas
redes = []

for n in num_nodos:
    for k in grados_medios:
        # El número de enlaces que se añaden desde un nuevo nodo a nodos existentes
        m = k // 2  # Se divide por 2 porque en el modelo Barabasi-Albert cada enlace añade 2 al grado total
        # Generar la red Barabasi-Albert
        G = nx.barabasi_albert_graph(n, m)
        redes.append(G)
        # Opcional: Visualizar la red
        #nx.draw(G, with_labels=False, node_size=30)
        #plt.show()

print(f"Se han generado {len(redes)} redes Barabasi-Albert.")


import networkx as nx
import matplotlib.pyplot as plt

# Parámetros para las redes
num_nodos = [2000, 4000]  # Número de nodos
grados_medios = [2, 4, 8]  # Grado medio de cada nodo

# Almacenar las redes generadas
redes_test = []

for n in num_nodos:
    for k in grados_medios:
        # El número de enlaces que se añaden desde un nuevo nodo a nodos existentes
        m = k // 2  # Se divide por 2 porque en el modelo Barabasi-Albert cada enlace añade 2 al grado total
        # Generar la red Barabasi-Albert
        G = nx.barabasi_albert_graph(n, m)
        redes_test.append(G)
        # Opcional: Visualizar la red
        #nx.draw(G, with_labels=False, node_size=30)
        #plt.show()

print(f"Se han generado {len(redes_test)} redes Barabasi-Albert.")


import networkx as nx

def guardar_redes(redes, prefijo):
    for i, G in enumerate(redes):
        nx.write_adjlist(G, f"{prefijo}_red_{i}.adjlist")

# Suponiendo que `redes` y `redes_test` son tus listas de redes
guardar_redes(redes, "train")
guardar_redes(redes_test, "test")


In [3]:
import networkx as nx
def cargar_redes(prefijo, num_redes):
    redes = []
    for i in range(num_redes):
        try:
            path = f"{prefijo}_red_{i}.adjlist"
            G = nx.read_adjlist(path)
            redes.append(G)
        except FileNotFoundError:
            print(f"No se encontró el archivo: {path}")
            break
    return redes

num_redes_train = 6  # Ajusta esto según el número de redes que generas
num_redes_test = 6  # Ajusta esto según el número de redes de prueba que generas

redes = cargar_redes("train", num_redes_train)
redes_test = cargar_redes("test", num_redes_test)

if not redes or not redes_test:
    print(f"hay algun problema.")


In [4]:
import networkx as nx
import numpy as np
import random

def simulate_SIR_and_generate_labels(G, initial_infecteds, p=0.2, r=0.02, max_steps=100):
    # Inicializar todos los nodos a susceptibles
    status = {node: 'S' for node in G.nodes()}

    # Infectar los nodos iniciales
    for node in initial_infecteds:
        status[node] = 'I'

    for step in range(max_steps):
        new_status = dict(status)
        for node in G.nodes():
            if status[node] == 'I':
                # Intentar infectar a los vecinos
                for neighbor in G.neighbors(node):
                    if status[neighbor] == 'S' and random.random() < p:
                        new_status[neighbor] = 'I'
                # Recuperar el nodo infectado
                if random.random() < r:
                    new_status[node] = 'R'
        if status == new_status:
            break  # Termina si no hay cambios
        status = new_status

    # Generar etiquetas basadas en el estado final de los nodos
    labels = [1 if status[node] == 'I' else 0 for node in G.nodes()]
    return labels



# Establecer el tamaño de nodos infectados iniciales a 30
initial_infected_size = 30
results_by_network_SIR = []

for G in redes:
    # Seleccionar nodos infectados iniciales basado en el tamaño de 30
    initial_infecteds = random.sample(list(G.nodes()), initial_infected_size)
    # Simular el modelo SIR
    results = simulate_SIR_and_generate_labels(G, initial_infecteds)
    # Guardar los resultados
    results_by_network_SIR.append(results)

In [5]:
def simulate_IC_and_generate_labels(G, initial_infecteds, p=0.2, max_steps=100):
    # Inicializar todos los nodos como susceptibles excepto los infectados iniciales
    status = {node: 'S' for node in G.nodes()}
    for node in initial_infecteds:
        status[node] = 'I'
    
    newly_infected = set(initial_infecteds)
    for step in range(max_steps):
        if not newly_infected:
            break  # Terminar si no hay nuevos infectados
        
        new_status = dict(status)
        current_newly_infected = set()
        for infected_node in newly_infected:
            for neighbor in G.neighbors(infected_node):
                if status[neighbor] == 'S' and random.random() < p:
                    new_status[neighbor] = 'I'
                    current_newly_infected.add(neighbor)
        
        status = new_status
        newly_infected = current_newly_infected
        
        if not newly_infected:
            break  # Termina si no hay nuevos infectados en este paso
    
    # Generar etiquetas basadas en el estado final de los nodos
    labels = [1 if status[node] == 'I' else 0 for node in G.nodes()]
    return labels


# Ahora vamos a generar las etiquetas con el modelo IC para todas las redes
# Establecer el tamaño de nodos infectados iniciales a 30
#initial_infected_size = 30
#results_by_network_IC = []

#for G in redes:
    # Seleccionar nodos infectados iniciales basado en el tamaño de 30
    #initial_infecteds = random.sample(list(G.nodes()), initial_infected_size)
    # Simular el modelo SIR
    #results = simulate_IC_and_generate_labels(G, initial_infecteds)
    # Guardar los resultados
    #results_by_network_IC.append(results)


In [6]:
import sys
sys.path.append('C:\\Users\\manue\\struc2vec')

In [7]:
import sys
sys.path.append('C:\\Users\\manue\\struc2vec\\src')

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from src.struc2vec import Graph
import networkx as nx

for i, G in enumerate(redes):
    nx.write_edgelist(G, f"red_sintetica_{i}.edgelist")

import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'  # Asumiendo que main.py está en el directorio src

for i, _ in enumerate(redes):
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}.edgelist --output embeddings_{i}.emb --dimensions 128 --walk-length 80 --num-walks 10 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")


In [9]:
import os

for i, _ in enumerate(redes):
    if os.path.isfile(f"embeddings_{i}.emb"):
        print(f"Los embeddings para la red {i} existen.")
    else:
        print(f"No se encontraron embeddings para la red {i}.")

Los embeddings para la red 0 existen.
Los embeddings para la red 1 existen.
Los embeddings para la red 2 existen.
Los embeddings para la red 3 existen.
Los embeddings para la red 4 existen.
Los embeddings para la red 5 existen.


In [10]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open('C:\\Users\\manue\\soc-wiki-Vote.mtx', 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [11]:
import networkx as nx
from torch_geometric.utils import from_networkx

# Crear un grafo vacío en NetworkX
G = nx.Graph()

# Leer las aristas desde el archivo 'fb-pages-food.edges'
with open('fb-pages-food.edges', 'r') as file:
    for line in file:
        if not line.startswith('%'):  # Ignorar líneas de comentarios
            parts = line.split(',')
            if len(parts) >= 2:
                source = int(parts[0])
                target = int(parts[1])
                G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [12]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open('C:\\Users\\manue\\web-edu.mtx', 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [13]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils import from_networkx

# Abrir el archivo y leer las líneas, omitiendo la primera línea de cabecera
with open('C:\\Users\\manue\\ia-fb-messages.mtx', 'r') as file:
    data = file.readlines()[1:]  # Ignora la primera línea con el comentario del MatrixMarket

# Preparar listas para filas y columnas
rows, cols = [], []

# Procesar cada línea excepto la cabecera
for line in data:
    if line.strip():  # asegurarse de que la línea no esté vacía
        parts = line.split()
        rows.append(int(parts[0]) - 1)  # Los índices en MTX están basados en 1, los convertimos a base 0
        cols.append(int(parts[1]) - 1)

# Crear un grafo en NetworkX
G = nx.Graph()
edges = zip(rows, cols)
G.add_edges_from(edges)

# Opcional: convertir a formato de PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [14]:
import torch
import networkx as nx
from torch_geometric.utils import from_networkx

# Nombre del archivo de texto
archivo = 'USairport500.txt'

# Crear un grafo en NetworkX
G = nx.Graph()

# Abrir el archivo y leer las líneas
with open(archivo, 'r') as file:
    for line in file:
        # Dividir la línea en sus componentes y convertirlos en enteros
        source, target, weight = map(int, line.split())
        # Agregar los nodos y las aristas al grafo
        G.add_edge(source, target, weight=weight)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [15]:
import networkx as nx
from torch_geometric.utils import from_networkx

# Ruta al archivo que contiene las aristas
archivo_aristas = 'soc-hamsterster.edges'

# Crear un grafo vacío
G = nx.Graph()

# Leer el archivo de aristas y agregarlas al grafo
with open(archivo_aristas, 'r') as file:
    for line in file:
        if not line.startswith('%'):  # Ignorar líneas de comentarios
            parts = line.split()
            if len(parts) >= 2:
                source = int(parts[0])
                target = int(parts[1])
                G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [16]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

# Crear un grafo vacío en NetworkX
G = nx.Graph()

# Leer el archivo CSV y agregar las aristas al grafo
with open('soc-sign-bitcoinalpha.csv', 'r') as file:
    for line in file:
        parts = line.strip().split(',')
        source = int(parts[0])
        target = int(parts[1])
        G.add_edge(source, target)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [17]:
import networkx as nx
from scipy.io import mmread
import torch
from torch_geometric.utils import from_networkx

# Leer el archivo MTX
matrix = mmread('econ-mahindas.mtx')

# Convertir la matriz dispersa en un grafo NetworkX
G = nx.Graph(matrix)

# Convertir el grafo a un objeto PyTorch Geometric
data = from_networkx(G)

# Agregar el nuevo grafo a la lista de redes de prueba
redes_test.append(data)


In [18]:
num_redes = len(redes_test)
print("Número de redes en redes_test:", num_redes)

Número de redes en redes_test: 14


from src.struc2vec import Graph
import networkx as nx

for i, G in enumerate(redes_test):
    nx.write_edgelist(G, f"red_sintetica_{i}_test.edgelist")

import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'  # Asumiendo que main.py está en el directorio src

# Ejecuta struc2vec solo para las primeras 6 redes
for i in range(6):
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}_test.edgelist --output embeddings_{i}_test.emb --dimensions 128 --walk-length 80 --num-walks 10 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")



from src.struc2vec import Graph
import networkx as nx
import subprocess

ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'

def save_edgelist(graph, filename):
    with open(filename, 'w') as file:
        for edge in graph.edge_index.T.tolist():
            file.write(f"{edge[0]} {edge[1]}\n")

indice_inicio = len(redes_test) - 8

for i in range(indice_inicio, len(redes_test)):
    save_edgelist(redes_test[i], f"red_sintetica_{i}_test.edgelist")


for i in range(indice_inicio, len(redes_test)):    
    comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{i}_test.edgelist --output embeddings_{i}_test.emb --dimensions 128 --walk-length 80 --num-walks 10 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
    resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)
    
    if resultado.returncode != 0:
        print(f"Error en la ejecución para la red {i}: {resultado.stderr}")
    else:
        print(f"Salida para la red {i}: {resultado.stdout}")


In [19]:
#from src.struc2vec import Graph
#import networkx as nx
#import subprocess

#ruta_python_env = 'C:\\Users\\manue\\anaconda3\\envs\\pyg_env\\python.exe'
#ruta_main_struc2vec = 'C:\\Users\\manue\\struc2vec\\src\\main.py'

#def save_edgelist(graph, filename):
#    with open(filename, 'w') as file:
#        for edge in graph.edge_index.T.tolist():
 #           file.write(f"{edge[0]} {edge[1]}\n")

## Índice de la red 7 en la lista redes_test
#indice_red_7 = 7

## Guardar la lista de adyacencia de la red 7
#save_edgelist(redes_test[indice_red_7], f"red_sintetica_{indice_red_7}_test.edgelist")

## Ejecutar struc2vec solo para la red 7
#comando = f"{ruta_python_env} {ruta_main_struc2vec} --input red_sintetica_{indice_red_7}_test.edgelist --output embeddings_{indice_red_7}_test.emb --dimensions 128 --walk-length 80 --num-walks 10 --workers 8 --OPT1 True --OPT2 True --OPT3 True"
#resultado = subprocess.run(comando, shell=True, capture_output=True, text=True)

#if resultado.returncode != 0:
#    print(f"Error en la ejecución para la red {indice_red_7}: {resultado.stderr}")
#else:
#    print(f"Salida para la red {indice_red_7}: {resultado.stdout}")


In [20]:
import os

for i, _ in enumerate(redes_test):
    if os.path.isfile(f"embeddings_{i}_test.emb"):
        print(f"Los embeddings para la red {i} existen.")
    else:
        print(f"No se encontraron embeddings para la red {i}.")


Los embeddings para la red 0 existen.
Los embeddings para la red 1 existen.
Los embeddings para la red 2 existen.
Los embeddings para la red 3 existen.
Los embeddings para la red 4 existen.
Los embeddings para la red 5 existen.
Los embeddings para la red 6 existen.
Los embeddings para la red 7 existen.
Los embeddings para la red 8 existen.
Los embeddings para la red 9 existen.
Los embeddings para la red 10 existen.
Los embeddings para la red 11 existen.
Los embeddings para la red 12 existen.
Los embeddings para la red 13 existen.


In [21]:
import torch
from torch_geometric.data import Data
import numpy as np
import networkx as nx

data_list_SIR = []  # Para almacenar objetos Data de todas las redes

for i in range(len(redes)):  # Asumimos que 'redes' contiene todas tus redes
    # Cargar la red sintética
    G = nx.read_edgelist(f'red_sintetica_{i}.edgelist', nodetype=int)
    
    # Cargar los embeddings generados por struc2vec
    embeddings = np.loadtxt(f'embeddings_{i}.emb', skiprows=1)
    node_indices = embeddings[:, 0].astype(int)  # No se usa directamente, pero útil para validar
    node_features = embeddings[:, 1:]  # Las características (embeddings) de los nodos
    
    # Crear un tensor de características
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Crear listas de aristas
    edge_list = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    
    # Generar etiquetas usando la función de simulación SIR o IC modificada
    # Aquí usamos simulate_SIR_and_generate_labels como ejemplo
    initial_infecteds = random.sample(list(G.nodes()), 30)  # Escoger 30 nodos al azar como infectados iniciales
    etiquetas = simulate_SIR_and_generate_labels(G, initial_infecteds)
    
    y = torch.tensor(etiquetas, dtype=torch.long)
    
    # Crear el objeto Data con x, edge_index y y
    data = Data(x=x, edge_index=edge_list, y=y)
    
    # Opcional: Crear y asignar train_mask aquí si es necesario
    # train_mask = torch.rand(len(etiquetas)) < 0.8  # Ejemplo de cómo crear un train_mask
    # data.train_mask = train_mask
    
    data_list_SIR.append(data)



In [22]:
import torch
from torch_geometric.data import Data
import numpy as np
import networkx as nx

data_list_IC = []  # Para almacenar objetos Data de todas las redes

for i in range(len(redes)):  # Asumimos que 'redes' contiene todas tus redes
    # Cargar la red sintética
    G = nx.read_edgelist(f'red_sintetica_{i}.edgelist', nodetype=int)
    
    # Cargar los embeddings generados por struc2vec
    embeddings = np.loadtxt(f'embeddings_{i}.emb', skiprows=1)
    node_indices = embeddings[:, 0].astype(int)  # No se usa directamente, pero útil para validar
    node_features = embeddings[:, 1:]  # Las características (embeddings) de los nodos
    
    # Crear un tensor de características
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Crear listas de aristas
    edge_list = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    
    # Generar etiquetas usando la función de simulación SIR o IC modificada
    # Aquí usamos simulate_SIR_and_generate_labels como ejemplo
    initial_infecteds = random.sample(list(G.nodes()), 30)  # Escoger 30 nodos al azar como infectados iniciales
    etiquetas = simulate_IC_and_generate_labels(G, initial_infecteds)
    
    y = torch.tensor(etiquetas, dtype=torch.long)
    
    # Crear el objeto Data con x, edge_index y y
    data = Data(x=x, edge_index=edge_list, y=y)
    
    # Opcional: Crear y asignar train_mask aquí si es necesario
    # train_mask = torch.rand(len(etiquetas)) < 0.8  # Ejemplo de cómo crear un train_mask
    # data.train_mask = train_mask
    
    data_list_IC.append(data)



In [23]:
import torch
from torch_geometric.nn import GCNConv, BatchNorm
from torch.nn import Dropout  # Corregido

class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = BatchNorm(hidden_dim)  # Batch Normalization después de la primera convolución
        self.conv2 = GCNConv(hidden_dim, hidden_dim)  # Añadimos una capa convolucional adicional
        self.bn2 = BatchNorm(hidden_dim)  # Batch Normalization después de la segunda convolución
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = BatchNorm(hidden_dim)  # Batch Normalization después de la segunda convolución
        self.conv4 = GCNConv(hidden_dim, output_dim)
        self.dropout = torch.nn.Dropout(0.4)  # Reducimos el Dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = self.bn1(x)  # Aplicamos Batch Normalization
        x = torch.relu(x)
        x = self.dropout(x)  # Aplicamos Dropout
        x = self.conv2(x, edge_index)
        x = self.bn2(x)  # Aplicamos Batch Normalization
        x = torch.relu(x)
        x = self.dropout(x)  # Aplicamos Dropout
        x = self.conv3(x, edge_index)
        x = self.bn3(x)  # Aplicamos Batch Normalization
        x = torch.relu(x)
        x = self.dropout(x)  # Aplicamos Dropout
        x = self.conv4(x, edge_index)
        
        
        
        return x


In [24]:
import torch

modelos_paths_GCN_SIR = []  # Lista para almacenar las rutas de los archivos de los modelos guardados

for idx, data in enumerate(data_list_SIR):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GNN(input_dim=x.size(1), hidden_dim=64, output_dim=2).to(device)
    data = data.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-1)
    criterion = torch.nn.CrossEntropyLoss()

    num_nodes = data.num_nodes
    train_mask = torch.rand(num_nodes) < 0.8
    val_mask = ~train_mask
    data.train_mask = train_mask
    data.val_mask = val_mask
    
     # Funciones para entrenar y validar
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        return loss

    def validate():
        model.eval()
        with torch.no_grad():
            out = model(data)
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        return val_loss

    for epoch in range(1000):
        train_loss = train()
        if (epoch + 1) % 5 == 0:
            val_loss = validate()
            if epoch == 999:  # Guarda los resultados en la última época
                print(f'Conjunto {idx+1}: Training Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}')
    
    # Guardar el modelo después del entrenamiento
    model_path = f'model_SIR_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GCN_SIR.append(model_path)
    print(f'Modelo SIR {idx+1} guardado en {model_path}')


Conjunto 1: Training Loss: 0.40359067916870117, Validation Loss: 0.36376675963401794
Modelo SIR 1 guardado en model_SIR_0.pt
Conjunto 2: Training Loss: 0.3915309011936188, Validation Loss: 0.4695465564727783
Modelo SIR 2 guardado en model_SIR_1.pt
Conjunto 3: Training Loss: 0.43096596002578735, Validation Loss: 0.4666045010089874
Modelo SIR 3 guardado en model_SIR_2.pt
Conjunto 4: Training Loss: 0.4799681305885315, Validation Loss: 0.4816197156906128
Modelo SIR 4 guardado en model_SIR_3.pt
Conjunto 5: Training Loss: 0.44698747992515564, Validation Loss: 0.4278711676597595
Modelo SIR 5 guardado en model_SIR_4.pt
Conjunto 6: Training Loss: 0.4273779094219208, Validation Loss: 0.40618520975112915
Modelo SIR 6 guardado en model_SIR_5.pt


In [25]:
import torch

modelos_paths_GCN_IC = []  # Lista para almacenar las rutas de los archivos de los modelos guardados

for idx, data in enumerate(data_list_IC):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GNN(input_dim=x.size(1), hidden_dim=64, output_dim=2).to(device)
    data = data.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-1)
    criterion = torch.nn.CrossEntropyLoss()

    num_nodes = data.num_nodes
    train_mask = torch.rand(num_nodes) < 0.8
    val_mask = ~train_mask
    data.train_mask = train_mask
    data.val_mask = val_mask
    
     # Funciones para entrenar y validar
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        return loss

    def validate():
        model.eval()
        with torch.no_grad():
            out = model(data)
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        return val_loss

    for epoch in range(1000):
        train_loss = train()
        if (epoch + 1) % 5 == 0:
            val_loss = validate()
            if epoch == 999:  # Guarda los resultados en la última época
                print(f'Conjunto {idx+1}: Training Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}')
    
    # Guardar el modelo después del entrenamiento
    model_path = f'model_IC_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GCN_IC.append(model_path)
    print(f'Modelo IC {idx+1} guardado en {model_path}')


Conjunto 1: Training Loss: 0.188833549618721, Validation Loss: 0.1882631778717041
Modelo IC 1 guardado en model_IC_0.pt
Conjunto 2: Training Loss: 0.5829439759254456, Validation Loss: 0.5624985694885254
Modelo IC 2 guardado en model_IC_1.pt
Conjunto 3: Training Loss: 0.6503745317459106, Validation Loss: 0.6663835644721985
Modelo IC 3 guardado en model_IC_2.pt
Conjunto 4: Training Loss: 0.1422957181930542, Validation Loss: 0.16148650646209717
Modelo IC 4 guardado en model_IC_3.pt
Conjunto 5: Training Loss: 0.48393163084983826, Validation Loss: 0.5227402448654175
Modelo IC 5 guardado en model_IC_4.pt
Conjunto 6: Training Loss: 0.6710789203643799, Validation Loss: 0.6695987582206726
Modelo IC 6 guardado en model_IC_5.pt


In [26]:
import random

def simular_difusion_SIR(G, semillas, p=0.05, r=0.02, max_steps=100):
    status = {node: 'S' for node in G.nodes()}
    for node in semillas:
        status[node] = 'I'
    
    #print(f"Estado inicial (semillas infectadas): {semillas}")

    for step in range(max_steps):
        new_status = dict(status)
        infectados = 0
        recuperados = 0
        for node in G.nodes():
            if status[node] == 'I':
                for neighbor in G.neighbors(node):
                    if status[neighbor] == 'S' and random.random() < p:
                        new_status[neighbor] = 'I'
                        infectados += 1
                if random.random() < r:
                    new_status[node] = 'R'
                    recuperados += 1
        status = new_status
        #print(f"Paso {step + 1}: Infectados nuevos = {infectados}, Recuperados = {recuperados}")
        
        if infectados == 0 and recuperados == 0:
            #print("No hay más cambios en el estado de los nodos.")
            break
    
    total_infectados = sum(1 for state in status.values() if state in ['I', 'R'])
    #print(f"Total infectados al final: {total_infectados}")
    return total_infectados


In [27]:
import numpy as np

def simular_difusion_IC(G, semillas, p=0.05, max_steps=100):
    status = {node: 'S' for node in G.nodes()}
    for node in semillas:
        if node not in G:
            print(f"El nodo {node} no está en el grafo.")
        status[node] = 'I'
    
    newly_infected = set(semillas)
    #print(f"Estado inicial (semillas infectadas): {semillas}")

    for step in range(max_steps):
        if not newly_infected:
     #       print("No hay nuevos infectados.")
            break
        
        new_status = dict(status)
        current_newly_infected = set()
        for infected_node in newly_infected:
            if infected_node not in G:
                continue  # Salta a la siguiente iteración si el nodo no existe
            for neighbor in G.neighbors(infected_node):
                if status[neighbor] == 'S' and np.random.random() < p:
                    new_status[neighbor] = 'I'
                    current_newly_infected.add(neighbor)
        
        status = new_status
        newly_infected = current_newly_infected
        #print(f"Paso {step + 1}: Nuevos infectados = {len(newly_infected)}")
        
    total_infectados = sum(1 for state in status.values() if state == 'I')
    #print(f"Total infectados al final: {total_infectados}")
    return total_infectados


In [28]:
def cargar_modelo_GCN_IC(i, input_dim, hidden_dim, output_dim, device):
    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    model.load_state_dict(torch.load(modelos_paths_GCN_IC[i]))
    model.to(device)
    return model

def cargar_modelo_GCN_SIR(i, input_dim, hidden_dim, output_dim, device):
    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    model.load_state_dict(torch.load(modelos_paths_GCN_SIR[i]))
    model.to(device)
    return model


def cargar_modelo_SIR(i, input_dim, hidden_dim, output_dim, device):
    model_path = modelos_paths_SIR[i]
    print(f"Cargando modelo SIR desde: {model_path}")  # Imprime la ruta del modelo
    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    # Imprimir detalles del modelo
    print("Detalles del modelo SIR cargado:")
    print(model)
    return model

def cargar_modelo_IC(i, input_dim, hidden_dim, output_dim, device):
    model_path = modelos_paths_IC[i]
    print(f"Cargando modelo IC desde: {model_path}")  # Imprime la ruta del modelo
    model = GNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    
    # Imprimir detalles del modelo
    print("Detalles del modelo IC cargado:")
    print(model)
    return model

modelo_sir = cargar_modelo_SIR(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
modelo_ic = cargar_modelo_IC(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Ahora imprime el state_dict para verificar nombres de las capas
print("Keys en el modelo SIR:")
print(list(modelo_sir.state_dict().keys()))
print("Keys en el modelo IC:")
print(list(modelo_ic.state_dict().keys()))


import numpy as np
import random

# Supongamos que 'G' es uno de tus grafos
G = redes_test[5]  # Usando el primer grafo como ejemplo

initial_infected_size = 30
initial_infecteds = random.sample(list(G.nodes()), initial_infected_size)

# Suponiendo que tienes funciones `simulate_SIR_and_generate_labels` y `simulate_IC_and_generate_labels` definidas
labels_SIR = simular_difusion_SIR(G, initial_infecteds)
labels_IC = simular_difusion_IC(G, initial_infecteds)

# Comparar las etiquetas generadas por ambos modelos
different_labels_count = np.sum(np.array(labels_SIR) != np.array(labels_IC))
print("Número de etiquetas diferentes entre SIR e IC:", different_labels_count)


# Asumiendo que el modelo y los datos están definidos en alguna parte del código
# Esto simplemente imprime un subconjunto de los datos usados para entrenar, ajusta según tu estructura de datos

print("Datos usados para entrenar el modelo SIR (primeras 10 etiquetas):", labels_SIR[:10])
print("Datos usados para entrenar el modelo IC (primeras 10 etiquetas):", labels_IC[:10])


# Cargar los modelos
modelo_sir = cargar_modelo_SIR(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
modelo_ic = cargar_modelo_IC(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Comparar los pesos de alguna capa después del entrenamiento
weights_SIR = modelo_sir.state_dict()['conv1.lin.weight'].cpu().numpy()
weights_IC = modelo_ic.state_dict()['conv1.lin.weight'].cpu().numpy()
print("Diferencia en pesos entre modelos SIR e IC:", np.linalg.norm(weights_SIR - weights_IC))


import matplotlib.pyplot as plt

def dibujar_grafo(grafo, title="Grafo", filename="grafo.png"):
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(grafo)
    nx.draw(grafo, pos, node_color='lightblue', with_labels=True, node_size=10, font_size=10, edge_color='gray')
    plt.title(title)
    plt.savefig(filename)
    plt.close()

for i, grafo in enumerate(redes):
    dibujar_grafo(grafo, f"Grafo de Entrenamiento {i}", f"entrenamiento_{i}.png")

for i, grafo in enumerate(redes_test):
    dibujar_grafo(grafo, f"Grafo de Prueba {i}", f"prueba_{i}.png")


import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch_geometric.utils import to_networkx

def dibujar_grafo_pyg(data, title="Grafo", filename=None):
    G = to_networkx(data, to_undirected=True)  # Convierte a grafo no dirigido
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)  # Genera una disposición de los nodos del grafo para una visualización agradable
    nx.draw(G, pos, node_color='lightblue', with_labels=True, node_size=500, font_size=10, edge_color='gray')
    plt.title(title)
    if filename:
        plt.savefig(filename)
    else:
        plt.show()

        
indice_inicio = 6
# Si tienes los objetos Data guardados en listas, por ejemplo `redes_test_pyg`
for i in range(indice_inicio, len(redes_test)):
    data = redes_test[i]
    dibujar_grafo_pyg(data, f"Grafo de Prueba PyG {i}", f"prueba_{i}.png")


def comparar_modelos(modelo_sir, modelo_ic):
    print("Comparando los modelos SIR e IC...")
    if modelo_sir.state_dict()['conv1.weight'].equal(modelo_ic.state_dict()['conv1.weight']):
        print("Los pesos de la primera capa convolucional son iguales.")
    else:
        print("Los pesos de la primera capa convolucional son diferentes.")

modelo_sir = cargar_modelo_SIR(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
modelo_ic = cargar_modelo_IC(0, input_dim=x.size(1), hidden_dim=64, output_dim=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

comparar_modelos(modelo_sir, modelo_ic)


import networkx as nx
import torch
from torch_geometric.utils import to_networkx

def check_and_convert_graph(graph, identifier=""):
    if isinstance(graph, Data):
        # Convertir de PyTorch Geometric Data a NetworkX Graph
        nx_graph = to_networkx(graph, to_undirected=True)
        print(f"Grafo {identifier} convertido de PyTorch Geometric a NetworkX.")
        return nx_graph
    elif isinstance(graph, nx.Graph):
        print(f"Grafo {identifier} ya es un objeto NetworkX.")
        return graph
    else:
        print(f"Error: El objeto {identifier} no es un grafo.")
        return None

for i, G_test in enumerate(redes_test):
    G_test = check_and_convert_graph(G_test, f"Test Grafo {i}")
    if G_test is None or len(G_test.edges()) == 0:
        print(f"No hay aristas válidas para el Grafo {i}")
        continue  # Salta este grafo si no tiene aristas

    # Procesamiento adicional aquí...


In [29]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

modelos_entrenados_SIR = [cargar_modelo_GCN_SIR(i, input_dim=128, hidden_dim=64, output_dim=2, device=device) for i in range(len(modelos_paths_GCN_SIR))]

resultados_influencia_SIR = np.zeros((len(modelos_entrenados_SIR), len(redes_test)))

for i, model in enumerate(modelos_entrenados_SIR):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_SIR[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_SIR = resultados_influencia_SIR.mean(axis=1)
mejor_red_idx_GCN_SIR = np.argmax(promedio_influencia_por_red_SIR)
print(f"La mejor red de entrenamiento SIR es la número {mejor_red_idx_GCN_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_SIR[mejor_red_idx_GCN_SIR]}")


La mejor red de entrenamiento SIR es la número 5, con un promedio de influencia normalizada de 7.523310964459225


In [30]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

modelos_entrenados_IC = [cargar_modelo_GCN_IC(i, input_dim=128, hidden_dim=64, output_dim=2, device=device) for i in range(len(modelos_paths_GCN_IC))]

resultados_influencia_IC = np.zeros((len(modelos_entrenados_IC), len(redes_test)))

for i, model in enumerate(modelos_entrenados_IC):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(device)

        model.eval()
        with torch.no_grad():
            out = model(data_test)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)
            total_infectados_red += infectados_un_tamano

        resultados_influencia_IC[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_IC = resultados_influencia_IC.mean(axis=1)
mejor_red_idx_GCN_IC = np.argmax(promedio_influencia_por_red_IC)
print(f"La mejor red de entrenamiento IC es la número {mejor_red_idx_GCN_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_IC[mejor_red_idx_GCN_IC]}")


La mejor red de entrenamiento IC es la número 2, con un promedio de influencia normalizada de 0.9025205877423507


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, num_heads):
        super(GATLayer, self).__init__()
        self.heads = nn.ModuleList([
            nn.Linear(in_features, out_features) for _ in range(num_heads)
        ])
        self.out_features = out_features
        self.num_heads = num_heads

    def forward(self, x, edge_index):
        # Concatenar las salidas de todas las cabezas de atención
        x = torch.cat([head(x) for head in self.heads], dim=1)
        # Aquí deberías añadir tu mecanismo de atención real usando edge_index
        return x

class GAT(nn.Module):
    def __init__(self, in_features, out_features, num_heads):
        super(GAT, self).__init__()
        self.gat_layer = GATLayer(in_features, out_features, num_heads)

    def forward(self, x, edge_index):
        x = self.gat_layer(x, edge_index)
        return x

# Ajusta 'in_features' a 128, que es la dimensión de tus características de entrada
model = GAT(in_features=128, out_features=32, num_heads=2)


In [32]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

for idx, data in enumerate(data_list_SIR):
    edge_indices = data.edge_index.cpu().numpy()  # Asegúrate de trabajar en la CPU
    max_index = edge_indices.max()
    min_index = edge_indices.min()
    num_nodes = data.num_nodes

    print(f"Data {idx}:")
    print(f"Shape of edge_index: {data.edge_index.shape}")
    print(f"Max index in edge_index: {max_index}")
    print(f"Min index in edge_index: {min_index}")
    print(f"Number of nodes (should be greater than max index): {num_nodes}")

    if max_index >= num_nodes:
        print("Error: Indices in edge_index are out of bounds!")
    if min_index < 0:
        print("Error: Negative indices found in edge_index!")


import torch
from torch_geometric.data import Data
import torch.optim as optim
import torch.nn.functional as F

class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Guarda el modelo cuando la pérdida de validación disminuya.'''
        self.val_loss_min = val_loss

In [33]:
import torch
from torch_geometric.data import Data
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Configura para usar la CPU y obtener mensajes de error más claros
device = torch.device('cpu')

# Suponiendo que tus modelos y datos están listos y que el modelo ya está definido en otro lugar como 'model'
modelos_paths_GAT_SIR = []

# Asegúrate de que cada Data object en data_list_SIR tenga train_mask, val_mask y test_mask
for data in data_list_SIR:
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Asignar aleatoriamente 80% de los nodos para entrenamiento, 10% para validación, 10% para prueba
    perm = torch.randperm(num_nodes)
    train_mask[perm[:int(0.8 * num_nodes)]] = True
    val_mask[perm[int(0.8 * num_nodes):int(0.9 * num_nodes)]] = True
    test_mask[perm[int(0.9 * num_nodes):]] = True

    # Agregar los masks al objeto data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def validate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.val_mask], data.y[data.val_mask])
    return loss.item()

# Asumiendo que el modelo y los datos ya están definidos
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

for idx, data in enumerate(data_list_SIR):
    data = data.to(device)
    model.to(device)
    best_val_loss = float('inf')

    for epoch in range(1000):
        train_loss = train(data)
        val_loss = validate(data)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}: Train Loss: {train_loss}, Val Loss: {val_loss}')


    # Siempre imprimir la mejor pérdida de validación al final del entrenamiento
    print(f'Best Val Loss: {best_val_loss}')
    model_path = f'model_GAT_SIR_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GAT_SIR.append(model_path)
    print(f'Modelo GAT SIR {idx+1} guardado en {model_path}')


Epoch 100: Train Loss: 0.42584773898124695, Val Loss: 0.3795829713344574
Epoch 200: Train Loss: 0.3892073929309845, Val Loss: 0.3596489429473877
Epoch 300: Train Loss: 0.37471500039100647, Val Loss: 0.36008384823799133
Epoch 400: Train Loss: 0.3656540811061859, Val Loss: 0.3611381947994232
Epoch 500: Train Loss: 0.3591890335083008, Val Loss: 0.36167553067207336
Epoch 600: Train Loss: 0.3543430268764496, Val Loss: 0.36209040880203247
Epoch 700: Train Loss: 0.35061511397361755, Val Loss: 0.36257439851760864
Epoch 800: Train Loss: 0.3476964831352234, Val Loss: 0.3631610572338104
Epoch 900: Train Loss: 0.3453789949417114, Val Loss: 0.3638227581977844
Epoch 1000: Train Loss: 0.34351590275764465, Val Loss: 0.3645162582397461
Best Val Loss: 0.35935071110725403
Modelo GAT SIR 1 guardado en model_GAT_SIR_0.pt
Epoch 100: Train Loss: 0.3892563581466675, Val Loss: 0.42190051078796387
Epoch 200: Train Loss: 0.3764994442462921, Val Loss: 0.4297636151313782
Epoch 300: Train Loss: 0.3696843087673187, 

In [34]:
import torch
from torch_geometric.data import Data
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Configura para usar la CPU y obtener mensajes de error más claros
device = torch.device('cpu')

# Suponiendo que tus modelos y datos están listos y que el modelo ya está definido en otro lugar como 'model'
modelos_paths_GAT_IC = []

# Asegúrate de que cada Data object en data_list_IC tenga train_mask, val_mask y test_mask
for data in data_list_IC:
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Asignar aleatoriamente 80% de los nodos para entrenamiento, 10% para validación, 10% para prueba
    perm = torch.randperm(num_nodes)
    train_mask[perm[:int(0.8 * num_nodes)]] = True
    val_mask[perm[int(0.8 * num_nodes):int(0.9 * num_nodes)]] = True
    test_mask[perm[int(0.9 * num_nodes):]] = True

    # Agregar los masks al objeto data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def validate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.val_mask], data.y[data.val_mask])
    return loss.item()

# Asumiendo que el modelo y los datos ya están definidos
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

for idx, data in enumerate(data_list_IC):
    data = data.to(device)
    model.to(device)
    best_val_loss = float('inf')

    for epoch in range(1000):
        train_loss = train(data)
        val_loss = validate(data)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}: Train Loss: {train_loss}, Val Loss: {val_loss}')


    # Siempre imprimir la mejor pérdida de validación al final del entrenamiento
    print(f'Best Val Loss: {best_val_loss}')
    model_path = f'model_GAT_IC_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GAT_IC.append(model_path)
    print(f'Modelo GAT IC {idx+1} guardado en {model_path}')


Epoch 100: Train Loss: 0.13900607824325562, Val Loss: 0.317048043012619
Epoch 200: Train Loss: 0.1257943958044052, Val Loss: 0.3092209994792938
Epoch 300: Train Loss: 0.11812405288219452, Val Loss: 0.3076991140842438
Epoch 400: Train Loss: 0.11331947147846222, Val Loss: 0.30812570452690125
Epoch 500: Train Loss: 0.11013782769441605, Val Loss: 0.30911415815353394
Epoch 600: Train Loss: 0.10794460028409958, Val Loss: 0.31023505330085754
Epoch 700: Train Loss: 0.1063888743519783, Val Loss: 0.31133440136909485
Epoch 800: Train Loss: 0.10526067763566971, Val Loss: 0.31235697865486145
Epoch 900: Train Loss: 0.10442707687616348, Val Loss: 0.3132849931716919
Epoch 1000: Train Loss: 0.10380098968744278, Val Loss: 0.3141140639781952
Best Val Loss: 0.30769097805023193
Modelo GAT IC 1 guardado en model_GAT_IC_0.pt
Epoch 100: Train Loss: 0.5557520389556885, Val Loss: 0.588675856590271
Epoch 200: Train Loss: 0.5395309329032898, Val Loss: 0.5881859064102173
Epoch 300: Train Loss: 0.5320850610733032, 

def cargar_modelo_SIR(i, in_features, out_features, num_heads, device):
    # Asegúrate de que estas dimensiones coincidan con las usadas durante el entrenamiento
    model = GAT(in_features=in_features, out_features=out_features, num_heads=num_heads)
    model.load_state_dict(torch.load(modelos_paths_SIR[i]))
    model.to(device)
    return model

def cargar_modelo_IC(i, in_features, out_features, num_heads, device):
    model = GAT(in_features=in_features, out_features=out_features, num_heads=num_heads)
    model.load_state_dict(torch.load(modelos_paths_IC[i]))  # Asegúrate que el path es correcto
    model.to(device)
    return model

In [35]:
def cargar_modelo_GAT_SIR(i, in_features, out_features, num_heads, device):
    model_path = modelos_paths_GAT_SIR[i]
    model = GAT(in_features=in_features, out_features=out_features, num_heads=num_heads)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    return model

def cargar_modelo_GAT_IC(i, in_features, out_features, num_heads, device):
    model_path = modelos_paths_GAT_IC[i]
    model = GAT(in_features=in_features, out_features=out_features, num_heads=num_heads)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    return model

In [36]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

# Asume que cargar_modelo_GAT está correctamente definida para cargar tu modelo GAT
modelos_entrenados_GAT = [
    cargar_modelo_GAT_SIR(i, in_features=128, out_features=32, num_heads=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    for i in range(len(modelos_paths_GAT_SIR))
]

resultados_influencia_GAT = np.zeros((len(modelos_entrenados_GAT), len(redes_test)))

for i, model in enumerate(modelos_entrenados_GAT):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)  # Asegúrate de tener una función de simulación adecuada para GAT
            total_infectados_red += infectados_un_tamano

        resultados_influencia_GAT[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_GAT = resultados_influencia_GAT.mean(axis=1)
mejor_red_idx_GAT_SIR = np.argmax(promedio_influencia_por_red_GAT)
print(f"La mejor red de entrenamiento GAT con SIR es la número {mejor_red_idx_GAT_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_GAT[mejor_red_idx_GAT_SIR]}")


La mejor red de entrenamiento GAT con SIR es la número 4, con un promedio de influencia normalizada de 7.631611210917664


In [37]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

# Asume que cargar_modelo_GAT está correctamente definida para cargar tu modelo GAT
modelos_entrenados_GAT = [
    cargar_modelo_GAT_IC(i, in_features=128, out_features=32, num_heads=2, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    for i in range(len(modelos_paths_GAT_IC))
]

resultados_influencia_GAT = np.zeros((len(modelos_entrenados_GAT), len(redes_test)))

for i, model in enumerate(modelos_entrenados_GAT):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)  # Asegúrate de tener una función de simulación adecuada para GAT
            total_infectados_red += infectados_un_tamano

        resultados_influencia_GAT[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_GAT = resultados_influencia_GAT.mean(axis=1)
mejor_red_idx_GAT_IC = np.argmax(promedio_influencia_por_red_GAT)
print(f"La mejor red de entrenamiento GAT con IC es la número {mejor_red_idx_GAT_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_GAT[mejor_red_idx_GAT_IC]}")


La mejor red de entrenamiento GAT con IC es la número 5, con un promedio de influencia normalizada de 0.8477902775127635


import torch
from torch_geometric.nn import SAGEConv, Sequential

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super(GraphSAGE, self).__init__()
        self.layers = Sequential('x, edge_index', [
            (SAGEConv(in_features, hidden_dim), 'x, edge_index -> x'),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.5),
            (SAGEConv(hidden_dim, hidden_dim), 'x, edge_index -> x'),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.5),
            (SAGEConv(hidden_dim, out_features), 'x, edge_index -> x')
        ])

    def forward(self, x, edge_index):
        x = self.layers(x, edge_index)
        return x


import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_features)
        self.dropout = nn.Dropout(p=0.5)  # Añade dropout con una probabilidad del 50%

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features, num_layers=3):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        # Capa de entrada
        self.convs.append(SAGEConv(in_features, hidden_dim))
        
        # Capas intermedias
        for i in range(1, num_layers-1):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        
        # Capa de salida
        self.convs.append(SAGEConv(hidden_dim, out_features))
        self.dropout = nn.Dropout(p=0.5)  # Añade dropout con una probabilidad del 50%
    
    def forward(self, x, edge_index):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        x = self.convs[-1](x, edge_index)
        return x


import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import Data

# Configura para usar la CPU o GPU según esté disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Lista para almacenar las rutas de los modelos entrenados
modelos_paths_SIR = []

# Asegúrate de que cada objeto Data en `data_list_SIR` tenga `train_mask`, `val_mask` y `test_mask`
for data in data_list_SIR:
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Asignar aleatoriamente 80% para entrenamiento, 10% para validación y 10% para prueba
    perm = torch.randperm(num_nodes)
    train_mask[perm[:int(0.8 * num_nodes)]] = True
    val_mask[perm[int(0.8 * num_nodes):int(0.9 * num_nodes)]] = True
    test_mask[perm[int(0.9 * num_nodes):]] = True

    # Agregar las máscaras al objeto data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

# Función para entrenar el modelo
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

# Función para validar el modelo
def validate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.val_mask], data.y[data.val_mask].to(device))
    return loss.item()

# Instancia de GraphSAGE
model = GraphSAGE(in_features=128, hidden_dim=64, out_features=2).to(device)

# Optimización y criterio de pérdida
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-2)
criterion = torch.nn.CrossEntropyLoss()

# Proceso de entrenamiento y validación para cada dataset en `data_list_SIR`
for idx, data in enumerate(data_list_SIR):
    data = data.to(device)
    best_val_loss = float('inf')

    # Entrenar durante un número determinado de épocas
    for epoch in range(1000):
        train_loss = train(data)
        val_loss = validate(data)

        # Guardar el mejor modelo basado en la validación
        if val_loss < best_val_loss:
            best_val_loss = val_loss

        # Reportar cada 100 épocas
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}')

    # Imprimir la mejor pérdida de validación al final
    print(f'Best Val Loss: {best_val_loss}')
    model_path = f'model_GraphSAGE_SIR_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_SIR.append(model_path)
    print(f'Modelo GraphSAGE SIR {idx + 1} guardado en {model_path}')


In [39]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import Data

# Configura para usar la CPU o GPU según esté disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Lista para almacenar las rutas de los modelos entrenados
modelos_paths_GraphSAGE_SIR = []

# Asegúrate de que cada objeto Data en `data_list_SIR` tenga `train_mask`, `val_mask` y `test_mask`
for data in data_list_SIR:
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Asignar aleatoriamente 80% para entrenamiento, 10% para validación y 10% para prueba
    perm = torch.randperm(num_nodes)
    train_mask[perm[:int(0.8 * num_nodes)]] = True
    val_mask[perm[int(0.8 * num_nodes):int(0.9 * num_nodes)]] = True
    test_mask[perm[int(0.9 * num_nodes):]] = True

    # Agregar las máscaras al objeto data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

# Función para entrenar el modelo
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

# Función para validar el modelo
def validate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.val_mask], data.y[data.val_mask].to(device))
    return loss.item()

# Instancia de GraphSAGE
model = GraphSAGE(in_features=128, hidden_dim=64, out_features=2, num_layers=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-3)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
criterion = nn.CrossEntropyLoss()

#patience = 150
#patience_counter = 0

# Proceso de entrenamiento y validación para cada dataset en `data_list_SIR`
for idx, data in enumerate(data_list_SIR):
    data = data.to(device)
    #best_val_loss = float('inf')

    # Entrenar durante un número determinado de épocas
    for epoch in range(1000):
        train_loss = train(data)
        val_loss = validate(data)
        #scheduler.step(val_loss)

        #if val_loss < best_val_loss:
         #   best_val_loss = val_loss
          #  patience_counter = 0  # reset counter if improvement is seen
        #else:
         #   patience_counter += 1

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}: Train Loss: {train_loss}, Val Loss: {val_loss}')    
        
        #if patience_counter >= patience:
         #   print("Early stopping triggered.")
          #  break


    #print(f'Best Val Loss: {best_val_loss}')
    model_path = f'model_GraphSAGE_SIR_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GraphSAGE_SIR.append(model_path)
    print(f'Modelo GraphSAGE SIR {idx+1} guardado en {model_path}')


Epoch 100: Train Loss: 0.3578612208366394, Val Loss: 0.4843791723251343
Epoch 200: Train Loss: 0.35700908303260803, Val Loss: 0.48934581875801086
Epoch 300: Train Loss: 0.35700035095214844, Val Loss: 0.48426494002342224
Epoch 400: Train Loss: 0.357101708650589, Val Loss: 0.4887193739414215
Epoch 500: Train Loss: 0.35730427503585815, Val Loss: 0.48866409063339233
Epoch 600: Train Loss: 0.3568708896636963, Val Loss: 0.4891139268875122
Epoch 700: Train Loss: 0.3579598367214203, Val Loss: 0.48832058906555176
Epoch 800: Train Loss: 0.3576521575450897, Val Loss: 0.48990967869758606
Epoch 900: Train Loss: 0.3568302392959595, Val Loss: 0.4892083406448364
Epoch 1000: Train Loss: 0.3593212068080902, Val Loss: 0.4809102714061737
Modelo GraphSAGE SIR 1 guardado en model_GraphSAGE_SIR_0.pt
Epoch 100: Train Loss: 0.3819368779659271, Val Loss: 0.4818965196609497
Epoch 200: Train Loss: 0.38176795840263367, Val Loss: 0.4810703992843628
Epoch 300: Train Loss: 0.3807585835456848, Val Loss: 0.482761621475

In [40]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import Data

# Configura para usar la CPU o GPU según esté disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Lista para almacenar las rutas de los modelos entrenados
modelos_paths_GraphSAGE_IC = []

# Asegúrate de que cada objeto Data en `data_list_IC` tenga `train_mask`, `val_mask` y `test_mask`
for data in data_list_IC:
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Asignar aleatoriamente 80% para entrenamiento, 10% para validación y 10% para prueba
    perm = torch.randperm(num_nodes)
    train_mask[perm[:int(0.8 * num_nodes)]] = True
    val_mask[perm[int(0.8 * num_nodes):int(0.9 * num_nodes)]] = True
    test_mask[perm[int(0.9 * num_nodes):]] = True

    # Agregar las máscaras al objeto data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

# Función para entrenar el modelo
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

# Función para validar el modelo
def validate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.val_mask], data.y[data.val_mask].to(device))
    return loss.item()

# Instancia de GraphSAGE
model = GraphSAGE(in_features=128, hidden_dim=64, out_features=2, num_layers=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-3)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
criterion = nn.CrossEntropyLoss()

#patience = 150
#patience_counter = 0

# Proceso de entrenamiento y validación para cada dataset en `data_list_IC`
for idx, data in enumerate(data_list_IC):
    data = data.to(device)
    #best_val_loss = float('inf')

    # Entrenar durante un número determinado de épocas
    for epoch in range(1000):
        train_loss = train(data)
        val_loss = validate(data)
        #scheduler.step(val_loss)

        #if val_loss < best_val_loss:
         #   best_val_loss = val_loss
          #  patience_counter = 0  # reset counter if improvement is seen
        #else:
         #   patience_counter += 1

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}: Train Loss: {train_loss}, Val Loss: {val_loss}')    
        
        #if patience_counter >= patience:
         #   print("Early stopping triggered.")
          #  break


    #print(f'Best Val Loss: {best_val_loss}')
    model_path = f'model_GraphSAGE_IC_{idx}.pt'
    torch.save(model.state_dict(), model_path)
    modelos_paths_GraphSAGE_IC.append(model_path)
    print(f'Modelo GraphSAGE IC {idx+1} guardado en {model_path}')


Epoch 100: Train Loss: 0.15050072968006134, Val Loss: 0.2367667257785797
Epoch 200: Train Loss: 0.15076451003551483, Val Loss: 0.23334266245365143
Epoch 300: Train Loss: 0.15054138004779816, Val Loss: 0.2335858792066574
Epoch 400: Train Loss: 0.1501556634902954, Val Loss: 0.23445402085781097
Epoch 500: Train Loss: 0.1484910249710083, Val Loss: 0.2349425107240677
Epoch 600: Train Loss: 0.14854800701141357, Val Loss: 0.23375597596168518
Epoch 700: Train Loss: 0.14917144179344177, Val Loss: 0.2331269383430481
Epoch 800: Train Loss: 0.14923995733261108, Val Loss: 0.23254740238189697
Epoch 900: Train Loss: 0.14931020140647888, Val Loss: 0.23521775007247925
Epoch 1000: Train Loss: 0.14880457520484924, Val Loss: 0.23389650881290436
Modelo GraphSAGE IC 1 guardado en model_GraphSAGE_IC_0.pt
Epoch 100: Train Loss: 0.5720323920249939, Val Loss: 0.5308099985122681
Epoch 200: Train Loss: 0.5716090202331543, Val Loss: 0.5311988592147827
Epoch 300: Train Loss: 0.5700216889381409, Val Loss: 0.52952486

In [41]:
def cargar_modelo_GraphSAGE_SIR(i, in_features, hidden_dim, out_features, num_layers, device):
    model = GraphSAGE(in_features, hidden_dim, out_features, num_layers)
    model_path = modelos_paths_GraphSAGE_SIR[i]
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    return model

def cargar_modelo_GraphSAGE_IC(i, in_features, hidden_dim, out_features, num_layers, device):
    model = GraphSAGE(in_features, hidden_dim, out_features, num_layers)
    model_path = modelos_paths_GraphSAGE_IC[i]
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    return model

In [42]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

# Carga los modelos entrenados
modelos_entrenados_SIR = [
    cargar_modelo_GraphSAGE_SIR(i, in_features=128, hidden_dim=64, out_features=2, num_layers=5, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    for i in range(len(modelos_paths_GraphSAGE_SIR))
]

resultados_influencia_GraphSAGE_SIR = np.zeros((len(modelos_entrenados_SIR), len(redes_test)))

for i, model in enumerate(modelos_entrenados_SIR):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_SIR(G_test, semillas)  # Asegúrate de tener una función de simulación adecuada
            total_infectados_red += infectados_un_tamano

        resultados_influencia_GraphSAGE_SIR[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_GraphSAGE_SIR = resultados_influencia_GraphSAGE_SIR.mean(axis=1)
mejor_red_idx_GraphSAGE_SIR = np.argmax(promedio_influencia_por_red_GraphSAGE_SIR)
print(f"La mejor red de entrenamiento GraphSAGE es la número {mejor_red_idx_GraphSAGE_SIR + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_GraphSAGE_SIR[mejor_red_idx_GraphSAGE_SIR]}")


La mejor red de entrenamiento GraphSAGE es la número 4, con un promedio de influencia normalizada de 7.427768984609267


In [43]:
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

def is_digit_string(s):
    """ Verifica si el objeto es un string que representa un dígito. """
    return isinstance(s, str) and s.isdigit()

# Carga los modelos entrenados
modelos_entrenados_IC = [
    cargar_modelo_GraphSAGE_IC(i, in_features=128, hidden_dim=64, out_features=2, num_layers=5, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    for i in range(len(modelos_paths_GraphSAGE_IC))
]

resultados_influencia_GraphSAGE_IC = np.zeros((len(modelos_entrenados_IC), len(redes_test)))

for i, model in enumerate(modelos_entrenados_IC):
    for j, data_test in enumerate(redes_test):
        if isinstance(data_test, Data):
            G_test = to_networkx(data_test, to_undirected=True)
        elif isinstance(data_test, nx.Graph):
            G_test = data_test
        else:
            print(f"Error: Grafo {j} no es compatible.")
            continue

        if not G_test.nodes or not G_test.edges:
            print(f"No hay nodos o aristas válidos para el Modelo {i}, Grafo {j}")
            continue

        # Convertir nodos a enteros si son strings numéricos
        mapping = {node: int(node) if is_digit_string(node) else node for node in G_test.nodes()}
        G_test = nx.relabel_nodes(G_test, mapping)

        embeddings_test = np.loadtxt(f'embeddings_{j}_test.emb', skiprows=1)
        x_test = torch.tensor(embeddings_test[:, 1:], dtype=torch.float)
        edge_index = torch.tensor(list(G_test.edges()), dtype=torch.long).t().contiguous()

        data_test = Data(x=x_test, edge_index=edge_index)
        data_test = data_test.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        model.eval()
        with torch.no_grad():
            out = model(data_test.x, data_test.edge_index)
            influencia_scores = out.softmax(dim=1)[:, 1]

        total_infectados_red = 0
        for size in range(10, 51, 5):
            _, indices_semillas = torch.topk(influencia_scores, size)
            semillas = [idx.item() for idx in indices_semillas if idx.item() in G_test.nodes()]

            if not semillas:
                print(f"No hay semillas válidas para el Modelo {i}, Grafo {j}, Tamaño {size}")
                continue

            infectados_un_tamano = simular_difusion_IC(G_test, semillas)  # Asegúrate de tener una función de simulación adecuada
            total_infectados_red += infectados_un_tamano

        resultados_influencia_GraphSAGE_IC[i, j] = total_infectados_red / G_test.number_of_nodes()

promedio_influencia_por_red_GraphSAGE_IC = resultados_influencia_GraphSAGE_IC.mean(axis=1)
mejor_red_idx_GraphSAGE_IC = np.argmax(promedio_influencia_por_red_GraphSAGE_IC)
print(f"La mejor red de entrenamiento GraphSAGE es la número {mejor_red_idx_GraphSAGE_IC + 1}, con un promedio de influencia normalizada de {promedio_influencia_por_red_GraphSAGE_IC[mejor_red_idx_GraphSAGE_IC]}")


La mejor red de entrenamiento GraphSAGE es la número 6, con un promedio de influencia normalizada de 0.8950303775940672


In [49]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import networkx as nx
from torch_geometric.data import Data

def seleccionar_semillas(modelo, data, size, model_type):
    modelo.eval()
    with torch.no_grad():
        if model_type == 'GCN':
            out = modelo(data)  # Para GNN que espera un objeto Data
        else:
            out = modelo(data.x, data.edge_index)  # Para GAT y GraphSAGE que esperan x y edge_index por separado
        influencia_scores = torch.softmax(out, dim=1)[:, 1]
        _, indices_semillas = torch.topk(influencia_scores, size)
        semillas = indices_semillas.tolist()
    return semillas

modelos = {
    'GCN': (cargar_modelo_GCN_SIR(mejor_red_idx_GCN_SIR, input_dim=128, hidden_dim=64, output_dim=2, device=device), 'GCN'),
    'GAT': (cargar_modelo_GAT_SIR(mejor_red_idx_GAT_SIR, in_features=128, out_features=32, num_heads=2, device=device), 'GAT'),
    'GraphSAGE': (cargar_modelo_GraphSAGE_SIR(mejor_red_idx_GraphSAGE_SIR, in_features=128, hidden_dim=64, out_features=2, num_layers=5, device=device), 'GraphSAGE')
}

sizes = range(10, 51, 5)

# Proceso para cada red en el rango de índices de 6 a 13
for index in range(6, 14):
    resultados = {modelo: [] for modelo in modelos}
    G_test = redes_test[index]
    G_test = to_networkx(G_test, to_undirected=True)
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})
    
    for modelo_name, (modelo, model_type) in modelos.items():
        for size in sizes:
            x = torch.tensor(np.random.rand(len(G_test.nodes()), 128), dtype=torch.float).to(device)
            edge_index = torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device)
            data_test = Data(x=x, edge_index=edge_index)
            
            semillas = seleccionar_semillas(modelo, data_test, size, model_type)
            infectados_total = 0
            num_runs = 10  # Número de simulaciones por tamaño de semilla para promediar
            for _ in range(num_runs):
                infectados = simular_difusion_SIR(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)
    
    # Graficación de los resultados para cada red
    plt.figure(figsize=(10, 6))
    for modelo_name, valores in resultados.items():
        plt.plot(list(sizes), valores, label=modelo_name, marker='o')
    plt.title(f'Influencia Promedio en SIR vs. Tamaño de Semillas Iniciales para Red {index}')
    plt.xlabel('Número de Semillas Iniciales')
    plt.ylabel('Influencia Promedio Normalizada')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'./resultados/Grafica_Influencia_SIR_vs_inicial_Red_{index}.png')
    plt.close()


In [50]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import networkx as nx
from torch_geometric.data import Data

def seleccionar_semillas(modelo, data, size, model_type):
    modelo.eval()
    with torch.no_grad():
        if model_type == 'GCN':
            out = modelo(data)  # Para GNN que espera un objeto Data
        else:
            out = modelo(data.x, data.edge_index)  # Para GAT y GraphSAGE que esperan x y edge_index por separado
        influencia_scores = torch.softmax(out, dim=1)[:, 1]
        _, indices_semillas = torch.topk(influencia_scores, size)
        semillas = indices_semillas.tolist()
    return semillas

modelos = {
    'GCN': (cargar_modelo_GCN_IC(mejor_red_idx_GCN_IC, input_dim=128, hidden_dim=64, output_dim=2, device=device), 'GCN'),
    'GAT': (cargar_modelo_GAT_IC(mejor_red_idx_GAT_IC, in_features=128, out_features=32, num_heads=2, device=device), 'GAT'),
    'GraphSAGE': (cargar_modelo_GraphSAGE_IC(mejor_red_idx_GraphSAGE_IC, in_features=128, hidden_dim=64, out_features=2, num_layers=5, device=device), 'GraphSAGE')
}

sizes = range(10, 51, 5)

# Proceso para cada red en el rango de índices de 6 a 13
for index in range(6, 14):
    resultados = {modelo: [] for modelo in modelos}
    G_test = redes_test[index]
    G_test = to_networkx(G_test, to_undirected=True)
    G_test = nx.relabel_nodes(G_test, {n: int(n) for n in G_test.nodes()})
    
    for modelo_name, (modelo, model_type) in modelos.items():
        for size in sizes:
            x = torch.tensor(np.random.rand(len(G_test.nodes()), 128), dtype=torch.float).to(device)
            edge_index = torch.tensor([[u, v] for u, v in G_test.edges()], dtype=torch.long).t().contiguous().to(device)
            data_test = Data(x=x, edge_index=edge_index)
            
            semillas = seleccionar_semillas(modelo, data_test, size, model_type)
            infectados_total = 0
            num_runs = 10  # Número de simulaciones por tamaño de semilla para promediar
            for _ in range(num_runs):
                infectados = simular_difusion_IC(G_test, semillas)
                infectados_total += infectados
            influencia_promedio = infectados_total / num_runs / len(G_test.nodes())
            resultados[modelo_name].append(influencia_promedio)
    
    # Graficación de los resultados para cada red
    plt.figure(figsize=(10, 6))
    for modelo_name, valores in resultados.items():
        plt.plot(list(sizes), valores, label=modelo_name, marker='o')
    plt.title(f'Influencia Promedio en IC vs. Tamaño de Semillas Iniciales para Red {index}')
    plt.xlabel('Número de Semillas Iniciales')
    plt.ylabel('Influencia Promedio Normalizada')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'./resultados/Grafica_Influencia_IC_vs_inicial_Red_{index}.png')
    plt.close()
